In [1]:
#requirment install

# !pip intstall tensorflow
# !pip intstall requests
# !pip intstall pandas numpy 


In [6]:
from tensorflow.keras.layers.experimental import preprocessing
import string 
import requests
import pandas as pd
import numpy as np
import os
import time
import tensorflow
import numpy as np 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras  import models
from tensorflow.keras.layers import Dense, LSTM, Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences


class CodeGeneration:
    X=[]
    y=[]
    seqXLength=0
    codeVocabSize=0
    modelLSTM=Sequential()
    df = pd.DataFrame() ;
    rawCodeList = [] ;
    EncodedCodes=[];
    token=Tokenizer(lower=False, filters='!"#$%&*+,-./:;<=>?@[\\]^_`{|}~\t\n');
    codeLineCoulmn="codes";
    defaultDatasetPath="data/top_poject_source_codes.csv";
    coustomDatasetLine=2000
    nGramcodeList=[]
    maxLengthPadding=40
    paddedCodeNgramSequences=[]
    predictCodeLength=2
    _Epochs=140
    _Batch_size=64
    _Use_saved_model=True;


 
    
    def __init__( self,lang, level,token=''):
        self.lang = lang
        self.level = level

   
    def loadCodeData(self):
        self.df=pd.read_csv(self.defaultDatasetPath)
        #         print(self.defaultDatasetPath)
        self.df=self.df[:self.coustomDatasetLine]
        return (self.df)
        

    def codeToRawCodeList(self):
        self.rawCodeList=self.df[self.codeLineCoulmn].tolist()
        return (self.rawCodeList)


    def getTotalWords(self):
        return len(" ".join(self.rawCodeList))
    
    def getWordFrequencyCounts(self):
        self.tokenizeCodes()
        self.encodeWords()
        return len(" ".join(self.rawCodeList))    
    
    def totalWords(self):
        return len(" ".join(self.rowcodeList))
    
    def tokenizeCodes(self):
        self.token.fit_on_texts(self.rawCodeList)
        self.EncodedCodes=self.token.texts_to_sequences(self.rawCodeList)
        return self.EncodedCodes
        
    def encodedWordsCount(self):
        self.tokenizeCodes();
        word_counts=self.token.word_counts
        return word_counts
        
    def encodedWordsIndex(self):
        self.tokenizeCodes();
        word_index=self.token.word_index
        return word_index   
    
    
    def encodedWordsIndex(self):
        self.tokenizeCodes();
        word_index=self.token.word_index
        return word_index   
    
    def getAllCodesWordSize(self):
        vocabSize=len(self.token.word_counts)+1
        return vocabSize     

    
    #break each line as ngram 
    def provideNgramSequences(self):
        nGramcodeList=[]

        for d in self.EncodedCodes:
            if len(d)>1:           
                for i in range(2,len(d)):
                    nGramcodeList.append(d[:i])
#                     print (d[:i])
        self.nGramcodeList=nGramcodeList            
        return nGramcodeList;
                    
    #provide padding input for ML or DL algorithms                
    def nGramcodeSeqPadding(self): 
        self.paddedCodeNgramSequences=pad_sequences(self.nGramcodeList, maxlen=self.maxLengthPadding,padding="pre")
        return self.paddedCodeNgramSequences
    
    
    def provideModelInputOutPut(self):
        self.X=self.paddedCodeNgramSequences[:,:-1] 
        self.y=self.paddedCodeNgramSequences[:,-1]
        
        #categorize y
        self.codeVocabSize=self.getAllCodesWordSize()
        self.y=to_categorical(self.y,num_classes=self.codeVocabSize)
        #X shape
        self.seqXLength=self.X.shape[1]
        
    
    
    
    
    def TrainLSTMModel(self):

        self.modelLSTM=Sequential()
        self.modelLSTM.add(Embedding(self.codeVocabSize,50,input_length=self.seqXLength))
        self.modelLSTM.add(LSTM(100,return_sequences=True))
        self.modelLSTM.add(LSTM(100))
        self.modelLSTM.add(Dense(100,activation="relu"))
        self.modelLSTM.add(Dense(self.codeVocabSize,activation="softmax"))
#         self.modelLSTM.summary()

        self.modelLSTM.compile(loss='categorical_crossentropy',optimizer='adam',metrics=["accuracy"])
        self.modelLSTM.fit(self.X,self.y, batch_size=self._Batch_size,epochs=self._Epochs)
        
        self.modelLSTM.save('saved_model/LSTM_code_generatiion_model')

        
        
    def plotModelResult(self):    
        import matplotlib.pyplot as plt
        
        if _Use_saved_model==True:
            return True;
        
        
    def loadSavedModel(self):

        isfile =os.path.exists(os.path.join(os.getcwd(), './saved_model/LSTM_code_generatiion_model', 'saved_model.pb'))
        if  isfile:     
            self.modelLSTM = tensorflow.keras.models.load_model('saved_model/LSTM_code_generatiion_model')
            return True
        
        return False
    


    def generate_code(self,seed_text,n_lines):
        
        self.loadCodeData()
        self.codeToRawCodeList()
        self.tokenizeCodes()
        self.encodedWordsCount()
        self.provideNgramSequences()
        self.nGramcodeSeqPadding()
        
        self.provideModelInputOutPut()

        isDir =os.path.isdir('./saved_model') 
        isfile =os.path.exists(os.path.join(os.getcwd(), './saved_model/LSTM_code_generatiion_model', 'saved_model.pb'))

        
        #load saved model
        if self._Use_saved_model==True and isDir and isfile:
                self.loadSavedModel()
        else:
                self.TrainLSTMModel()
        
        predictionList= []    
        seq_length=self.seqXLength

        for i in range(n_lines):
            text=[]
            for _ in range(self.predictCodeLength):
                encoded=self.token.texts_to_sequences([seed_text])
                encoded=pad_sequences(encoded,maxlen=seq_length,padding='pre')

                y_pred=np.argmax( self.modelLSTM.predict(encoded),axis=1)

                #find to word dictinary which word mapped number
                predicted_code=""
                for word,index in self.token.word_index.items():
                    if index== y_pred:
                        predicted_code=word
                        break
                seed_text=seed_text +' '+ predicted_code
                text.append(predicted_code)

            seed_text=text[-1]
            text=' '.join(text) 
            print(text)
            predictionList.append(text)
        
        return   predictionList
    
    
    
    

In [7]:
#sample Code
codeGenObj=CodeGeneration('Java','line')

#sample1
seed_text='import'
predictionList=codeGenObj.generate_code(seed_text,1)


android support


In [14]:
#sample 2

seed_text='package'
predictionList=codeGenObj.generate_code(seed_text,2)

com viewpagerindicator
measureSpec packed
